In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps


# Import API key
from api_keys import weather_api_key
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [16]:
#load in the csv file
cities_weather = pd.read_csv("output/weather_df.csv", encoding="utf-8")
cities_weather.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed
0,0,faanui,100,PF,1594613130,78,-16.48,-151.75,78.51,12.62
1,1,kathmandu,75,NP,1594613131,88,27.72,85.32,73.40,1.66
2,2,mataura,83,NZ,1594613131,87,-46.19,168.86,43.75,3.13
3,3,sao felix do xingu,92,IN,1594613131,71,20.51,76.21,80.51,12.55
4,4,katsuura,100,JP,1594613131,82,35.13,140.30,73.00,1.99


In [17]:
#find the humidity and locations based on lat and lng
humidity = cities_weather['Humidity']
max_humidity = humidity.max()
city = cities_weather[["Lat", "Lng"]]

In [18]:
#create gmap figure
gmap_figure = gmaps.figure()
heat_layer_map = gmaps.heatmap_layer(city, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
gmap_figure.add_layer(heat_layer_map)
gmap_figure

Figure(layout=FigureLayout(height='420px'))

In [23]:
#create new df Hotel and filter out based on criteria 
hotel_df = cities_weather.loc[(cities_weather["Temperature"] > 70) & (cities_weather["Temperature"] < 80) & (cities_weather["Cloudiness"] == 0), :]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
hotel_df.head()

#add a new column
hotel_df['Hotel Name'] = ""

# params for the loop
params = {
    "radius": 5000,
    "types": "Hotel",
    "location": f"{lat},{lng}",
    "key": g_key
}
# loop to find each hotel
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.head()

,index,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Wind Speed,Hotel Name
0,24,24,port alfred,0,CD,1594613137,56,-7.03,29.76,70.41,3.89,Kirungu
1,49,49,carnarvon,0,AU,1594613142,57,-24.87,113.63,77.00,5.82,Carnarvon
2,65,65,pacific grove,0,US,1594613146,86,38.48,-90.74,75.99,4.59,Wildwood
3,86,86,puerto ayora,0,ES,1594613152,83,36.59,-6.23,75.99,6.93,El Puerto de Santa María
4,93,93,port macquarie,0,CD,1594613154,56,-7.03,29.76,70.41,3.89,Kirungu


In [24]:

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:

markers = gmaps.marker_layer(locations)
gmap_figure.add_layer(markers)
gmap_figure

Figure(layout=FigureLayout(height='420px'))